In [6]:
from ioh import get_problem, ProblemClass
from ioh import logger
import math
import os
from dataclasses import dataclass
import BA 
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.random_projection import GaussianRandomProjection
import botorch
import gpytorch
from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, ScaleKernel, RBFKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch.quasirandom import SobolEngine

from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.exceptions import ModelFittingError
from botorch.fit import fit_gpytorch_mll
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from sklearn.preprocessing import StandardScaler


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")
dtype = torch.float
SMOKE_TEST = os.environ.get("SMOKE_TEST")

# problem dimension
dim = 40 if not SMOKE_TEST else 50
projected_dim = 15

# initial sample points
n_init = 10 if not SMOKE_TEST else 4
max_cholesky_size = float("inf")

problem = get_problem(1, dimension=dim, instance=2, problem_class=ProblemClass.BBOB)

Running on cpu


In [8]:
EVALUATION_BUDGET = 100 if not SMOKE_TEST else 10
NUM_RESTARTS = 3 if not SMOKE_TEST else 2
RAW_SAMPLES = 51 if not SMOKE_TEST else 4
N_CANDIDATES = min(dim, max(200, 20 * dim)) if not SMOKE_TEST else 4

In [13]:
S = BA.embedding_matrix_morenonzero(7, 3)
X = torch.randint(100, (7, 3))
print(f"S before increase\n{S}")
print(f"X before increase\n{X}")

S before increase
tensor([[-1.,  0., -1.,  1., -1.,  1.,  0.],
        [ 0.,  1., -1.,  1.,  0., -1.,  1.],
        [-1.,  1.,  0.,  0.,  1.,  0.,  1.]])
X before increase
tensor([[ 3, 89, 72],
        [71, 38, 69],
        [70, 80, 89],
        [38, 87, 47],
        [57, 95, 76],
        [93, 44,  1],
        [25, 70, 38]])


In [14]:
S, X = BA.increase_embedding_and_observations(S, X, 3)
print(f"S after increase\n{S}")
print(f"X after increase\n{X}")

S after increase
tensor([[ 0.,  0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  1.],
        [-1.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [-1.,  0.,  0.,  0.,  0.,  0.,  0.]])
X after increase
tensor([[ 3, 89, 72,  3,  3, 89, 89, 72, 72],
        [71, 38, 69, 71, 71, 38, 38, 69, 69],
        [70, 80, 89, 70, 70, 80, 80, 89, 89],
        [38, 87, 47, 38, 38, 87, 87, 47, 47],
        [57, 95, 76, 57, 57, 95, 95, 76, 76],
        [93, 44,  1, 93, 93, 44, 44,  1,  1],
        [25, 70, 38, 25, 25, 70, 70, 38, 38]])


In [5]:
state = BA.BaxusState(dim=dim, eval_budget=EVALUATION_BUDGET - n_init)
n_sample = 40
S = BA.embedding_matrix_random(input_dim=state.dim, target_dim=state.d_init)
X_baxus_target = BA.get_initial_points(state.d_init, n_init)
X_baxus_input = X_baxus_target @ S
Y_baxus = torch.tensor(
        [-problem(list(x)) for x in X_baxus_input], dtype=dtype, device=device
    ).unsqueeze(-1)

with botorch.settings.validate_input_scaling(False):
    for _ in range(EVALUATION_BUDGET - n_init):  # Run until evaluation budget depleted
            # Fit a GP model
            train_Y = (Y_baxus - Y_baxus.mean()) / Y_baxus.std()
            likelihood = GaussianLikelihood(noise_constraint=Interval(1e-8, 1e-3))
            model = SingleTaskGP(
                X_baxus_target, train_Y, likelihood=likelihood
            )
            print(model.covar_module.lengthscale)

            mll = ExactMarginalLogLikelihood(model.likelihood, model)

            # Fit the model using Cholesky context
            with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                try:
                    fit_gpytorch_mll(mll)
                except ModelFittingError:
                    # Use Adam-based optimization if Cholesky decomposition fails
                    optimizer = torch.optim.Adam([{"params": model.parameters()}], lr=0.1)
                    for _ in range(200):
                        optimizer.zero_grad()
                        output = model(X_baxus_target)
                        loss = -mll(output, train_Y.flatten())
                        loss.backward()
                        optimizer.step()

                # Generate new candidates
                X_next_target = BA.create_candidate(
                    state=state,
                    model=model,
                    X=X_baxus_target,
                    Y=train_Y,
                    device=device,
                    dtype=dtype,
                    n_candidates=N_CANDIDATES,
                    num_restarts=NUM_RESTARTS,
                    raw_samples=RAW_SAMPLES,
                    acqf="ts",
                )
            
            # Map new candidates to high-dimensional space
            X_next_input = X_next_target @ S
            Y_next = torch.tensor(
                [-problem(list(x)) for x in X_next_input], dtype=dtype, device=device
            ).unsqueeze(-1)

            # Update state and concatenate new points
            state = BA.update_state(state=state, Y_next=Y_next)
            X_baxus_input = torch.cat((X_baxus_input, X_next_input), dim=0)
            X_baxus_target = torch.cat((X_baxus_target, X_next_target), dim=0)
            Y_baxus = torch.cat((Y_baxus, Y_next), dim=0)
            
            print(
                f"iteration {len(X_baxus_input)}, d={len(X_baxus_target.T)})  Best value: {state.best_value:.3}, TR length: {state.length:.3}"
            )

            if state.restart_triggered:
                state.restart_triggered = False
                print("increasing target space")

                S, X_baxus_target = BA.increase_embedding_and_observations(
                    S, X_baxus_target, state.new_bins_on_split
                )
                print(f"new dimensionality: {len(S)}")
                
                state.target_dim = len(S)
                state.length = state.length_init
                state.failure_counter = 0
                state.success_counter = 0


Parameter containing:
tensor([[0.2048]], requires_grad=True)


/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_7421/761235705.py:15: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_7421/761235705.py:15: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability

iteration 11, d=1)  Best value: -6.75e+02, TR length: 0.4
Parameter containing:
tensor([[0.2048]], requires_grad=True)
iteration 12, d=1)  Best value: -6.75e+02, TR length: 0.2
Parameter containing:
tensor([[0.2048]], requires_grad=True)
iteration 13, d=1)  Best value: -6.75e+02, TR length: 0.1
Parameter containing:
tensor([[0.2048]], requires_grad=True)
iteration 14, d=1)  Best value: -6.75e+02, TR length: 0.05
Parameter containing:
tensor([[0.2048]], requires_grad=True)
iteration 15, d=1)  Best value: -6.75e+02, TR length: 0.025
Parameter containing:
tensor([[0.2048]], requires_grad=True)


/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_7421/761235705.py:15: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision an

iteration 16, d=1)  Best value: -6.75e+02, TR length: 0.0125
Parameter containing:
tensor([[0.2048]], requires_grad=True)
iteration 17, d=1)  Best value: -6.75e+02, TR length: 0.00625
increasing target space
new dimensionality: 2
Parameter containing:
tensor([[0.2896, 0.2896]], requires_grad=True)
iteration 18, d=2)  Best value: -6.68e+02, TR length: 0.8
Parameter containing:
tensor([[0.2896, 0.2896]], requires_grad=True)
iteration 19, d=2)  Best value: -6.68e+02, TR length: 0.4
Parameter containing:
tensor([[0.2896, 0.2896]], requires_grad=True)
iteration 20, d=2)  Best value: -6.65e+02, TR length: 0.4
Parameter containing:
tensor([[0.2896, 0.2896]], requires_grad=True)


...

/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10

iteration 84, d=16)  Best value: -5.99e+02, TR length: 0.8
Parameter containing:
tensor([[0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191,
         0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191]],
       requires_grad=True)
iteration 85, d=16)  Best value: -5.98e+02, TR length: 0.8
Parameter containing:
tensor([[0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191,
         0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191]],
       requires_grad=True)
iteration 86, d=16)  Best value: -5.96e+02, TR length: 0.8
Parameter containing:
tensor([[0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191,
         0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191]],
       requires_grad=True)
iteration 87, d=16)  Best value: -5.96e+02, TR length: 0.8
Parameter containing:
tensor([[0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191,
         0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191]],
   

/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10

iteration 89, d=16)  Best value: -5.95e+02, TR length: 0.8
Parameter containing:
tensor([[0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191,
         0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191, 0.8191]],
       requires_grad=True)


KeyboardInterrupt: 